<a href="https://colab.research.google.com/github/Shkum/IT_STEP/blob/ML/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%97_8_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn import metrics

from sklearn import tree

In [2]:
df1 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv",
                 index_col="Unnamed: 0")

In [3]:
df1.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [4]:
df1 = df1[['pclass', 'age', 'embarked', 'fare']]
df1.head()

,pclass,age,embarked,fare
0,3,22.0,S,7.2500
1,1,38.0,C,71.2833
2,3,26.0,S,7.9250
3,1,35.0,S,53.1000
4,3,35.0,S,8.0500


# Завдання 1
Очистіть дані від викидів(лише `fare`), розділіть на тренувальну та тестову чатини

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    891 non-null    int64  
 1   age       714 non-null    float64
 2   embarked  889 non-null    object 
 3   fare      891 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 34.8+ KB


In [6]:
def remove_outliers(df, column_name):
    column = df[column_name]
    q1 = column.quantile(q=0.25)
    q3 = column.quantile(q=0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    return df[(column >= lower) & (column <= upper)]

df = remove_outliers(df1, 'fare')



In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    775 non-null    int64  
 1   age       613 non-null    float64
 2   embarked  775 non-null    object 
 3   fare      775 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 30.3+ KB


In [8]:
y = df['fare']
X = df.drop(columns='fare')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Завдання 2
Створіть Pipeline для обробки даних

In [10]:
num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns


num_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median"))]
            )

cat_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="most_frequent")),
                   ("encoder", OneHotEncoder())])

preprocessor = ColumnTransformer(
        transformers=[
                ("num", num_transformer, num_columns),
                ("cat", cat_transformer, cat_columns), ])

print(preprocessor)

preprocessor.fit(X_train)
preprocessor.transform(X_train)



ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['pclass', 'age'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['embarked'], dtype='object'))])


array([[ 3.,  9.,  0.,  0.,  1.],
       [ 3., 28.,  0.,  0.,  1.],
       [ 3., 22.,  1.,  0.,  0.],
       ...,
       [ 2., 30.,  1.,  0.,  0.],
       [ 3., 28.,  0.,  1.,  0.],
       [ 3., 22.,  0.,  0.,  1.]])

# Завдання 3
Створіть остаточну модель та натренуйте її

In [11]:
model = Pipeline([("prep", preprocessor),
                  ("model", tree.DecisionTreeRegressor(max_depth=4, random_state=42))])

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=4, random_state=42))])

In [12]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=4, random_state=42))])

# Завдання 4
Виведіть основні метрики моделі

In [13]:
def get_metrics(y_true, y_pred, name="model"):
    df = pd.DataFrame()
    df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
    df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
    df.loc["R2", name] = metrics.r2_score(y_true, y_pred)
    return df.round(2)

y_pred = model.predict(X_test)


all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

all_metrics

,train,test
MAE,6.27,8.57
RMSE,8.77,13.27
R2,0.53,0.30


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

Попробуйте:
* не видаляти викиди
* змінити обробку даних(imputer та кодування категоріальних даних)
* змінити параметри дерева(глибина, кількість точок у листках тощо

In [14]:
# do not remove ouliets
y = df1['fare']
X = df1.drop(columns='fare')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns

num_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])

cat_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),
("encoder", OneHotEncoder())])

preprocessor = ColumnTransformer(

transformers=[("num", num_transformer, num_columns),
              ("cat", cat_transformer, cat_columns), ])

print(preprocessor)

preprocessor.fit(X_train)

preprocessor.transform(X_train)

model = Pipeline([("prep", preprocessor),
                  ("model", tree.DecisionTreeRegressor(max_depth=4, random_state=42))])

print(model)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

all_metrics

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['pclass', 'age'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['embarked'], dtype='object'))])
Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                 

,train,test
MAE,16.40,21.30
RMSE,34.92,43.57
R2,0.55,-0.23


In [15]:
# change points at leafs
model = Pipeline([("prep", preprocessor),
                  ("model", tree.DecisionTreeRegressor(max_depth=4, random_state=42, min_samples_leaf=5))])

print(model)

model.fit(X_train, y_train)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=4, min_samples_leaf=5,
                                       random

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=4, min_samples_leaf=5,
                                       random_state=42))])

In [16]:
y_pred = model.predict(X_test)

all_metrics = pd.DataFrame()
all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

all_metrics

,train,test
MAE,16.63,21.00
RMSE,35.45,42.67
R2,0.53,-0.18


In [17]:
# change depth
model = Pipeline([("prep", preprocessor),
                  ("model", tree.DecisionTreeRegressor(max_depth=12, random_state=42))])

print(model)

model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=12, random_state=42))])


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=12, random_state=42))])

In [18]:
y_pred = model.predict(X_test)

all_metrics = pd.DataFrame()
all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

all_metrics



,train,test
MAE,11.28,21.50
RMSE,24.61,51.94
R2,0.78,-0.74


# Завдання 6
Збережіть модель

In [19]:
import joblib

joblib.dump(model, 'my_model')

['my_model']